<a href="https://colab.research.google.com/github/MuhammedShiway/AICrowd-Mosquito/blob/main/%5B03%5D_YOLO_Det_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
data_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train_images'
train_csv_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train.csv'
yolo_train = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train'
yolo_test = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Test'



---

# <center> IMPORT MODULES AND LIBRARIES </center>


---



In [3]:
import os
import shutil
import csv



---

# <center> YOLO V8 Training </center>


---



In [4]:
# Paths
source_folder = data_path
train_folder = yolo_train
test_folder = yolo_test
csv_file = train_csv_path

In [5]:
# SETUP

%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [6]:
%cd /content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training

/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training


In [7]:
!ls

data.yaml  runs  yolov8l.pt  yolov8n.pt


In [ ]:
#TRAIN THE MODEL

!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=200 imgsz=640 plots=True

100% 21.5M/21.5M [00:00<00:00, 46.3MB/s]
Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False